<a href="https://colab.research.google.com/github/Abs-CV/Text-Generation-Word-Level-Neural-Language-Model/blob/main/Word_Level_Neural_Language_Model_to_Generate_Text_PLATO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

Saving republic.txt to republic.txt


In [3]:
def load_doc(filename):
  file = open(filename,'r')
  text=file.read()
  file.close()
  return(text)

In [4]:
in_filename='republic.txt'
doc=load_doc(in_filename)
print(doc[:200])

BOOK I.


I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in wha


In [5]:
import string

def clean_doc(doc):
  #replace '--' with a space
  doc = doc.replace('--',' ')
  #split into token with white spaces
  tokens = doc.split()
  #remove punctuation from each string
  table=str.maketrans('','',string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  #remove remaining tokens that are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]
  #make lower case
  tokens = [word.lower() for word in tokens]
  return tokens

In [6]:
#clean document
tokens=clean_doc(doc)
print(tokens[:200])
print(len(tokens),len(set(tokens)))

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid',

In [7]:
length =50+1
sequences=list()
for i in range(length,len(tokens)):
  #select sequence of tokens
  seq=tokens[i-length:i]
  #convert it to a line
  line = ' '.join(seq)
  #store
  sequences.append(line)

len(sequences),sequences[0] , sequences[1] 

(117291,
 'book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was',
 'i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted')

In [8]:
def save_doc(lines,filename):
  data='\n'.join(lines)
  file = open(filename,'w')
  file.write(data)
  file.close

In [9]:
out_file='writi.txt'
save_doc(sequences,out_file)

In [10]:
lines=sequences[:]

In [11]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [12]:
lines[0]

'book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was'

In [13]:
vocab_size=len(tokenizer.word_index)+1
vocab_size

7324

In [14]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence

In [15]:
#seperate into input and output
from keras.utils import to_categorical
sequences = np.array(sequences)
X,y=sequences[:,:-1],sequences[:,-1]
y= to_categorical(y,num_classes=vocab_size)
seq_length = X.shape[1]
seq_length

50

In [16]:
sequences[:,:-1].shape,y.shape,X.shape

((117291, 50), (117291, 7324), (117291, 50))

In [17]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            366200    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7324)              739724    
                                                                 
Total params: 1,256,824
Trainable params: 1,256,824
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
917/917 [==============================] - 20s 14ms/step - loss: 6.1427 - accuracy: 0.0739
Epoch 2/100
917/917 [==============================] - 13s 14ms/step - loss: 5.6857 - accuracy: 0.1077
Epoch 3/100
917/917 [==============================] - 13s 14ms/step - loss: 5.4429 - accuracy: 0.1315
Epoch 4/100
917/917 [==============================] - 13s 14ms/step - loss: 5.2884 - accuracy: 0.1455
Epoch 5/100
917/917 [==============================] - 13s 14ms/step - loss: 5.1801 - accuracy: 0.1544
Epoch 6/100
917/917 [==============================] - 14s 15ms/step - loss: 5.0873 - accuracy: 0.1598
Epoch 7/100
917/917 [==============================] - 14s 15ms/step - loss: 5.0743 - accuracy: 0.1588
Epoch 8/100
917/917 [==============================] - 14s 15ms/step - loss: 4.9518 - accuracy: 0.1686
Epoch 9/100
917/917 [==============================] - 13s 15ms/step - loss: 4.8534 - accuracy: 0.1753
Epoch 10/100
917/917 [==============================] - 13s 14ms/step - l

In [ ]:
# save the model to file
model.save('model.h5')

In [20]:
# save the tokenizer
from pickle import dump
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [28]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = np.argmax(model.predict(encoded), axis=1)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [29]:
from random import randint
from pickle import load
from keras.models import load_model
from keras_preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model


# load cleaned text sequences
in_filename = 'writi.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

make us look downwards and not upwards what do you mean he asked you i replied have in your mind a truly sublime conception of our knowledge of the things above and i dare say that if a person were to throw his head back and study the fretted ceiling you

1/1 [==============================] - 0s 16ms/step
would still think that the just man is derived from the object which they require who virtuous will profit thither in the same time highspirited and powerful of them poetry boldly intelligence but the inharmonious is cowardly and boorish very true and when a man feel right he replied you
